In [1]:
!pip install -q transformers==4.41.2


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:

!pip install -q sentence-transformers==2.2.2
#!pip install -q xformers==0.0.23
!pip install -q chromadb==0.4.20


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd

# Loading the Dataset

Using: kotartemiy/topic-labeled-news-dataset from Kaggle

In [8]:

import zipfile

# Define the path to your zip file
file_path = './data/topic-labeled-news-dataset.zip'

In [9]:
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('./data/')

In [18]:
# Using MAX_NEWS to limit the data for our limited hardware
news = pd.read_csv('./data/labelled_newscatcher_dataset.csv', sep=';')
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"
subset_news = news.head(MAX_NEWS)

ChromaDB requires that the data has a unique identifier. You can achieve it with the statement below, which will create a new column called Id.

In [13]:
news["id"] = news.index
news.head(3)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2


# ChromaDB

In [14]:
import chromadb
from chromadb.config import Settings

In [15]:
chroma_client = chromadb.PersistentClient(path="./db")

In [16]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)
collection = chroma_client.create_collection(name=collection_name)

In [19]:
collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

2024-08-15 09:41:33.375829 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-08-15 09:41:33.376229 [W:onnxruntime:, coreml_execution_provider.cc:104 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 49 number of nodes in the graph: 323 number of nodes supported by CoreML: 231


# Filling and Querying the ChromaDB Database

In [20]:
from datetime import datetime

In [21]:
collection_name = "news_collection"+datetime.now().strftime("%s")
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)

In [22]:
collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

In [24]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id173', 'id829', 'id117', 'id535', 'id141', 'id218', 'id390', 'id273', 'id56', 'id900']], 'distances': [[0.8593513369560242, 1.029434323310852, 1.0793386697769165, 1.092977523803711, 1.1329509019851685, 1.2130101919174194, 1.2143404483795166, 1.2163968086242676, 1.2220265865325928, 1.2753719091415405]], 'metadatas': [[{'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}, {'topic': 'TECHNOLOGY'}]], 'embeddings': None, 'documents': [['The Legendary Toshiba is Officially Done With Making Laptops', '3 gaming laptop deals you can’t afford to miss today', 'Lenovo and HP control half of the global laptop market', 'Asus ROG Zephyrus G14 gaming laptop announced in India', 'Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865)', "Apple's Next MacBook

# Vector Map

In [26]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [27]:
getado = collection.get(ids="id141",
                       include=["documents", "embeddings"])

In [28]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

[[-0.08085300773382187,
  -0.04996924847364426,
  -0.023777848109602928,
  -0.01105515193194151,
  0.026662329211831093,
  -0.04479314759373665,
  -0.028894929215312004,
  0.026656920090317726,
  0.0014408831484615803,
  -0.016409534960985184,
  0.06534290313720703,
  -0.06901559978723526,
  -0.057479437440633774,
  0.010111069306731224,
  0.050431180745363235,
  -0.0020670080557465553,
  0.07256121933460236,
  -0.12437129765748978,
  0.010660870932042599,
  -0.10942178964614868,
  -0.011435404419898987,
  -0.010370778851211071,
  -0.020607078447937965,
  -0.02439030632376671,
  0.07829038798809052,
  0.005822318606078625,
  0.023322677239775658,
  -0.08244234323501587,
  -0.027263037860393524,
  0.004673689138144255,
  0.004337726626545191,
  0.03252646327018738,
  -0.026033250615000725,
  0.0796395093202591,
  0.04218673333525658,
  -0.12119845300912857,
  0.04906683787703514,
  -0.07625890523195267,
  0.04331577941775322,
  -0.08360300958156586,
  -0.071403369307518,
  -0.0187938101

# Loading the model and creating the prompt

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

In [32]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

# Creating the extended prompt

In [30]:
question = "Can I buy a new Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"""
Relevant context: {context}
Considering the relevant context, answer the question.
Question: {question}
Answer: """
prompt_template

"\nRelevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot\nConsidering the relevant context, answer the question.\nQuestion: Can I buy a new Toshiba laptop?\nAnswer: "

In [33]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])


Relevant context: #The Legendary Toshiba is Officially Done With Making Laptops #3 gaming laptop deals you can’t afford to miss today #Lenovo and HP control half of the global laptop market #Asus ROG Zephyrus G14 gaming laptop announced in India #Acer Swift 3 featuring a 10th-generation Intel Ice Lake CPU, 2K screen, and more launched in India for INR 64999 (US$865) #Apple's Next MacBook Could Be the Cheapest in Company's History #Features of Huawei's Desktop Computer Revealed #Redmi to launch its first gaming laptop on August 14: Here are all the details #Toshiba shuts the lid on laptops after 35 years #This is the cheapest Windows PC by a mile and it even has a spare SSD slot
Considering the relevant context, answer the question.
Question: Can I buy a new Toshiba laptop?
Answer: 
Based on the given material, the answer to the question is no. Toshiba has discontinued making laptops.
